In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import altair as alt
import panel as pn
from function import *

C:\Users\shank\anaconda3\envs\message1.1\lib\site-packages\xarray\core\merge.py:10: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version



# **PlotResults**

## Read the CSV

In [2]:
# name of the CSV
n_file = 'GOA_Baseline.csv'      # file name
location = 'Output_Data'               # file address 
n_file_r = pd.read_csv(location +'/' + n_file, low_memory=False)

## Name of techs

Read the excel file that contains the name of technologies and the alias or easy names

In [3]:
#names of tech  The file Tech_names contains a dictionary with the techs and easy names for the Goa model

#names of tech elecetrcity
elec_en = pd.read_excel('Tech_names_Goa.xlsx', sheet_name='Power', usecols='A,B,C,D')
#names of other sectors
fe_en = pd.read_excel('Tech_names_Goa.xlsx', sheet_name='sectoral_mix', usecols='A,D,E,F')
#tn names of all tech joined for emissions
emi_en = pd.concat([elec_en,fe_en], ignore_index=True)
# alias for primary names
pe_en = pd.read_excel('Tech_names_Goa.xlsx', sheet_name='primary_e', usecols='A,B,C')

C:\Users\shank\anaconda3\envs\message1.1\lib\site-packages\ipykernel_launcher.py:8: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [4]:
#sectorial mix
sector_names = pd.read_excel('Tech_names_Goa.xlsx', usecols='A, B,C,D,E,F', sheet_name='sectoral_mix')

sector_namesT = sector_names[sector_names['Sector'].isin(['Public','Private','Freight'])] # to use in rate of use by tech for the transport
sector_namesOTH = sector_names[~sector_names['Sector'].isin(['Public','Private','Freight'])]

In [5]:
colors = pd.concat([sector_names,pe_en,emi_en])[['Tech_name','Sector','Fuel','Color']]
fuel_colors = colors[['Fuel','Color']].drop_duplicates().set_index('Fuel')
sec_colors = colors[['Sector','Color']].drop_duplicates().set_index('Sector')
tec_colors = colors[['Tech_name','Color']].drop_duplicates().set_index('Tech_name')

fuel_colors = fuel_colors[~fuel_colors.index.duplicated()].to_dict()['Color']
sec_colors = sec_colors[~sec_colors.index.duplicated(keep='first')].to_dict()['Color']
tec_colors = tec_colors[~tec_colors.index.duplicated(keep='first')].to_dict()['Color']

C:\Users\shank\anaconda3\envs\message1.1\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




## Reading csv file and segregating

In [6]:
act = n_file_r.loc[n_file_r.NAME=='ProdByTech']
pe = n_file_r.loc[n_file_r.NAME=='ProdAnn']
new_cap = n_file_r.loc[n_file_r.NAME=='NewCapacity']
tot_cap = n_file_r.loc[n_file_r.NAME=='TotCapacityAnn']
dem = n_file_r.loc[n_file_r.NAME=='RateOfUseByTech']
aet = n_file_r.loc[n_file_r.NAME=='AnnTechEmission']
inv = n_file_r.loc[n_file_r.NAME=='DiscCapitalInvestment']

# Primary Energy

In [7]:
sply = act.copy()
sply = (pd.merge(sply, pe_en[['TECHNOLOGY', 'primary_names','Color']] , on='TECHNOLOGY'))
sply = sply.groupby(['YEAR','primary_names','Color'], as_index=False).sum()
sply = sply[sply['VALUE'] > 0]
base = alt.Chart(sply.dropna(),width=614,height=437)
selection = alt.selection_multi(fields=['primary_names'],bind='legend')
cmap = sply[['primary_names','Color']].drop_duplicates()
grouped = base.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Primary Energy in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('primary_names', scale=alt.Scale(domain=cmap['primary_names'].values.tolist(), range=cmap['Color'].values.tolist())),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['primary_names','YEAR','sum(VALUE)']
            ).add_selection(selection).interactive()
text = base.mark_text(dx=0,dy=-40, color='black').encode(
    x=alt.X('YEAR:N', stack='zero'),
    y=alt.Y('sum(VALUE)'),
    text=alt.Text('sum(VALUE)', format='.2f')
)

# Each sector plot filtered by clicking the stacked plot
one_group = grouped.mark_bar().encode(
    alt.Y("sum(VALUE)",title='Primary Energy in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('primary_names', scale=alt.Scale(domain=cmap['primary_names'].values.tolist(), range=cmap['Color'].values.tolist())),
    tooltip=['primary_names','sum(VALUE)','YEAR'],
    row='primary_names',
).transform_filter( 
    selection
).interactive()


In [8]:
# (grouped)&one_group

In [9]:
# base.mark_line(dx=0,dy=-40, color='black').encode(
#     x=alt.X('YEAR:N', stack='zero'),
#     y=alt.Y('sum(VALUE)',title='Primary Energy in PJ'),
#     tooltip=['sum(VALUE)','YEAR'],
# )

# Final Energy by Fuel and Sectors

In [10]:
trp = dem.loc[dem['TECHNOLOGY'].isin(np.array(sector_namesT['TECHNOLOGY']))]
trp = rate_us_ts(trp)
trp = (pd.merge(trp, sector_namesT[['TECHNOLOGY', 'Tech_name','Sector','Fuel','Color','Sector_Color']] , on='TECHNOLOGY'))
oth = (pd.merge(act.copy(), sector_namesOTH[['TECHNOLOGY', 'Tech_name','Sector','Fuel','Color','Sector_Color']] , on='TECHNOLOGY'))

final_energy_all = oth.append(trp, ignore_index=True)
final_energy_fuel = final_energy_all.groupby(['YEAR','Fuel','Color'], as_index=False).sum()
final_energy_sec = final_energy_all.groupby(['YEAR','Sector','Sector_Color'], as_index=False).sum()

final_energy_fuel = final_energy_fuel[final_energy_fuel['VALUE'] > 0]
final_energy_sec = final_energy_sec[final_energy_sec['VALUE'] > 0]

fe_fuel = alt.Chart(final_energy_fuel.dropna(),width=614,height=437)
fe_sec = alt.Chart(final_energy_sec.dropna(),width=614,height=437)

selection = alt.selection_multi(fields=['Fuel'],bind='legend')
cmap = final_energy_fuel[['Fuel','Color']].drop_duplicates()

fe_by_fuel = fe_fuel.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Final Energy in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Fuel', scale=alt.Scale(domain=cmap['Fuel'].values.tolist(), range=cmap['Color'].values.tolist())),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['Fuel','YEAR','sum(VALUE)']
            ).add_selection(selection).interactive()
one_fuel = fe_by_fuel.mark_area().encode(
    alt.Y("sum(VALUE)",title='Final Energy in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('Fuel', scale=alt.Scale(domain=cmap['Fuel'].values.tolist(), range=cmap['Color'].values.tolist())),
    tooltip=['Fuel','sum(VALUE)','YEAR'],
    row='Fuel',
).transform_filter( 
    selection
).interactive()


cmap = final_energy_sec[['Sector','Sector_Color']].drop_duplicates()
fe_by_sec = fe_sec.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Final Energy in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Sector', scale=alt.Scale(domain=cmap['Sector'].values.tolist(), range=cmap['Sector_Color'].values.tolist())),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['Sector','YEAR','sum(VALUE)']
            ).add_selection(selection).interactive()


text = fe_fuel.mark_text(dx=0,dy=-40, color='black').encode(
    x=alt.X('YEAR:N', stack='zero'),
    y=alt.Y('sum(VALUE)'),
    text=alt.Text('sum(VALUE)', format='.2f')
)



# Each sector plot filtered by clicking the stacked plot

one_sec = fe_by_sec.mark_area().encode(
    alt.Y("sum(VALUE)",title='Final Energy in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('Sector', scale=alt.Scale(domain=cmap['Sector'].values.tolist(), range=cmap['Sector_Color'].values.tolist())),
    tooltip=['Sector','sum(VALUE)','YEAR'],
    row='Sector',
).transform_filter( 
    selection
).interactive()

C:\Users\shank\OneDrive\Documents\Github\Goa\scripts\function.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\shank\OneDrive\Documents\Github\Goa\scripts\function.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\shank\anaconda3\envs\message1.1\lib\site-packages\pandas\core\frame.py:4097: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [11]:
# fe_by_fuel&one_fuel
# fe_by_sec&one_sec

## Emissions

In [12]:
#Emissions

#annual emission by tech
elec_emi = (pd.merge(aet.copy(), elec_en, on='TECHNOLOGY'))
elec_emi['Sector'] = elec_emi['Tech_name']
elec_emi.rename({'Color': 'Sector_Color'}, axis=1,inplace=True)
fe_emi = (pd.merge(aet.copy(), fe_en, on='TECHNOLOGY'))
fe_emi.drop(columns=['Color'],inplace=True)
fe_emi["Sector"] = [sector[0] for sector in fe_emi['Tech_name'].str.split('-')]
all_emi = pd.concat([elec_emi[["Tech_name",'Sector','YEAR','Sector_Color','VALUE']],fe_emi[["Tech_name",'Sector','YEAR','Sector_Color','VALUE']]])

In [13]:
emissions = alt.Chart(all_emi.dropna(),width=614,height=437)
selection = alt.selection_multi(fields=['Sector'],bind='legend')
cmap = all_emi[['Sector','Sector_Color']].drop_duplicates()
tot_emi_by_year = emissions.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Emissions in TTCO2e',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Sector', scale=alt.Scale(domain=cmap['Sector'].values.tolist(), range=cmap['Sector_Color'].values.tolist())),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['YEAR','Sector','sum(VALUE)']
            ).add_selection(selection).interactive()
# Each sector plot filtered by clicking the stacked plot
one_emi = tot_emi_by_year.mark_bar().encode(
    alt.Y("sum(VALUE)",title='Emissions in TTCO2e',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('Sector', scale=alt.Scale(domain=cmap['Sector'].values.tolist(), range=cmap['Sector_Color'].values.tolist())),
    tooltip=['Tech_name','sum(VALUE)','YEAR'],
    row='Sector',
).transform_filter( 
    selection
).interactive()

In [14]:
# tot_emi_by_year&one_emi

# COST

## CAP INVESTMENT BY SECTOR

In [15]:
trp_corr = pd.read_excel('Transport_correction.xlsx').set_index('Tech_name').to_dict()

In [16]:
elec_inv = (pd.merge(inv.copy(), elec_en, on='TECHNOLOGY'))
elec_inv['Sector'] = elec_inv['Tech_name']
elec_inv.rename({'Color': 'Sector_Color'}, axis=1,inplace=True)
fe_inv = (pd.merge(inv.copy(), fe_en, on='TECHNOLOGY'))
fe_inv.drop(columns=['Color'],inplace=True)
fe_inv["Sector"] = [sector[0] for sector in fe_inv['Tech_name'].str.split('-')]
all_inv = pd.concat([elec_inv[["Tech_name",'Sector','YEAR','Sector_Color','VALUE']],fe_inv[["Tech_name",'Sector','YEAR','Sector_Color','VALUE']]])

In [17]:
for key,value in trp_corr['km'].items():
    all_inv.loc[all_inv['Tech_name'].str.contains(key+'-'), 'VALUE'] /= value


In [18]:
capinv = alt.Chart(all_inv.dropna(),width=614,height=437)
selection = alt.selection_multi(fields=['Sector'],bind='legend')
cmap = all_inv[['Sector','Sector_Color']].drop_duplicates()
tot_inv_by_year = capinv.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='New Investments in INR Crore',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Sector', scale=alt.Scale(domain=cmap['Sector'].values.tolist(), range=cmap['Sector_Color'].values.tolist())),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['YEAR','Sector','sum(VALUE)']
            ).add_selection(selection).interactive()

# Each sector plot filtered by clicking the stacked plot
one_inv = tot_inv_by_year.mark_bar().encode(
    alt.Y("sum(VALUE)",title='New Investments in INR Crore',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.X("Tech_name",title='Technology',axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
    alt.Color('Sector', scale=alt.Scale(domain=cmap['Sector'].values.tolist(), range=cmap['Sector_Color'].values.tolist())),
    tooltip=['Tech_name','sum(VALUE)','YEAR'],
    row='Sector',
).transform_filter( 
    selection
).interactive()

In [19]:
# tot_inv_by_year & one_inv

# Sector Mix

# Electricity Generation

In [20]:
#Production by tech electrcity

p_tech = act.copy()
#only for the electricity sector
p_tech = p_tech.loc[p_tech['FUEL'].isin(np.array(elec_en['Electr']))]
p_tech = (pd.merge(p_tech, elec_en[['TECHNOLOGY', 'Tech_name','Color']] , on='TECHNOLOGY'))
p_tech = p_tech.groupby(['YEAR','Tech_name', 'TIMESLICE','Color'], as_index=False).sum() 
# p_tech.drop(p_tech.columns[4:], axis=1, inplace=True)
p_tech = p_tech[["YEAR","Tech_name","TIMESLICE","Color","VALUE"]]

In [21]:
generation = alt.Chart(p_tech.dropna(),width=614,height=437)
selection = alt.selection_multi(fields=['Tech_name'],bind='legend')
cmap = p_tech[['Tech_name','Color']].drop_duplicates()
generation_by_year = generation.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Electricity Generation in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Tech_name', scale=alt.Scale(domain=cmap['Tech_name'].values.tolist(), range=cmap['Color'].values.tolist())),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['YEAR','Tech_name','sum(VALUE)']
            ).add_selection(selection).interactive()
# Each sector plot filtered by clicking the stacked plot
one_year = generation_by_year.mark_bar().encode(
    alt.Y("sum(VALUE)",title='Electricity Generation in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.X("TIMESLICE:N",title='Electricity Generation in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('Tech_name', scale=alt.Scale(domain=cmap['Tech_name'].values.tolist(), range=cmap['Color'].values.tolist())),
    tooltip=['Tech_name','sum(VALUE)','YEAR'],
    row='YEAR',
).transform_filter( 
    selection
).interactive()


In [22]:
# generation_by_year&one_year

## Total Capacity

In [23]:
all_cap =  tot_cap.copy()
all_cap = all_cap.loc[all_cap['TECHNOLOGY'].isin(np.array(elec_en['TECHNOLOGY']))]
all_cap = (pd.merge(all_cap, elec_en[['TECHNOLOGY', 'Tech_name','Color']] , on='TECHNOLOGY'))
all_cap = all_cap.groupby(['YEAR','Tech_name','Color'], as_index=False).sum()
all_cap.drop(all_cap[all_cap.Tech_name == 'IEX-Imports'].index, inplace=True)
all_cap.drop(all_cap[all_cap.Tech_name == 'PPA(New/Renewed)'].index, inplace=True)
all_cap = all_cap[["YEAR","Tech_name","Color","VALUE"]] 

In [24]:

totcap = alt.Chart(all_cap.dropna(),width=614,height=437)
selection = alt.selection_multi(fields=['Tech_name'],bind='legend')
cmap = p_tech[['Tech_name','Color']].drop_duplicates()

tot_cap_by_year = totcap.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Electricity Capacity in GW',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Tech_name', scale=alt.Scale(domain=cmap['Tech_name'].values.tolist(), range=cmap['Color'].values.tolist())),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['YEAR','Tech_name','sum(VALUE)']
            ).add_selection(selection).interactive()
# Each sector plot filtered by clicking the stacked plot
one_cap = tot_cap_by_year.mark_bar().encode(
    alt.Y("sum(VALUE)",title='Electricity Generation in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('Tech_name', scale=alt.Scale(domain=cmap['Tech_name'].values.tolist(), range=cmap['Color'].values.tolist())),
    tooltip=['Tech_name','sum(VALUE)','YEAR'],
    row='Tech_name',
).transform_filter( 
    selection
).interactive()

In [25]:

# tot_cap_by_year&one_cap

## New Capacity

In [26]:
# New capacity

ncap = new_cap.copy()
ncap = ncap.loc[ncap['TECHNOLOGY'].isin(np.array(elec_en['TECHNOLOGY']))]
ncap = (pd.merge(ncap, elec_en[['TECHNOLOGY', 'Tech_name',"Color"]] , on='TECHNOLOGY'))
ncap = ncap.groupby(['YEAR','Tech_name',"Color"], as_index=False).sum()
ncap.drop(ncap[ncap.Tech_name == 'IEX-Imports'].index, inplace=True)
ncap.drop(ncap[ncap.Tech_name == 'PPA(New/Renewed)'].index, inplace=True)
ncap = ncap[["YEAR","Tech_name","Color","VALUE"]]

In [27]:
newcap = alt.Chart(ncap.dropna(),width=614,height=437)
selection = alt.selection_multi(fields=['Tech_name'],bind='legend')
cmap = p_tech[['Tech_name','Color']].drop_duplicates()

new_cap_by_year = newcap.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Electricity Capacity in GW',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Tech_name', scale=alt.Scale(domain=cmap['Tech_name'].values.tolist(), range=cmap['Color'].values.tolist())),
                opacity = alt.condition(selection, alt.value(1), alt.value(0.1)),
                tooltip=['YEAR','Tech_name','sum(VALUE)']
            ).add_selection(selection).interactive()
# Each sector plot filtered by clicking the stacked plot
one_cap = new_cap_by_year.mark_bar().encode(
    alt.Y("sum(VALUE)",title='Electricity Generation in PJ',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
    alt.Color('Tech_name', scale=alt.Scale(domain=cmap['Tech_name'].values.tolist(), range=cmap['Color'].values.tolist())),
    tooltip=['Tech_name','sum(VALUE)','YEAR'],
    row='Tech_name',
).transform_filter( 
    selection
).interactive()

In [28]:
# new_cap_by_year&one_cap

In [46]:
import panel as pn
sectors = final_energy_all.Sector.drop_duplicates().tolist()
def fe_chart(sector):
    fe_dem = final_energy_all.loc[final_energy_all.Sector==sector,['YEAR','Tech_name',"Color",'VALUE']].groupby(['YEAR','Tech_name','Color']).sum().reset_index()
    cmap = fe_dem[['Tech_name','Color']].drop_duplicates()
    sec_chart = alt.Chart(fe_dem.dropna(),width=614,height=437)
    chart = sec_chart.mark_area().encode(
                alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
                alt.Y("sum(VALUE)",title='Electricity Capacity in GW',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
                alt.Color('Tech_name', scale=alt.Scale(domain=cmap['Tech_name'].values.tolist(), range=cmap['Color'].values.tolist())),
                tooltip=['YEAR','Tech_name','sum(VALUE)']
            )
    return pn.pane.Vega(chart)
     
pn.interact(fe_chart,sector=sectors).show()

Launching server at http://localhost:54421


In [30]:
sectors

['Agriculture-pump',
 'Agriculture-mecanization',
 'Commercial',
 'Fisheries',
 'Industry',
 'Cooking urban',
 'Cooking rural',
 'Residential',
 'Public',
 'Private',
 'Freight']

In [34]:
fe_dem = final_energy_all.loc[final_energy_all.Sector=='Private',['YEAR','Tech_name',"Color",'VALUE']]
cmap = fe_dem[['Tech_name','Color']].drop_duplicates()
sec_chart = alt.Chart(fe_dem.dropna(),width=614,height=437)
chart = sec_chart.mark_area().encode(
            alt.X("YEAR:N",axis=alt.Axis(labelFontSize=15,labelAngle=270,titleFontSize=20)),
            alt.Y("sum(VALUE)",title='Electricity Capacity in GW',axis=alt.Axis(labelFontSize=15,labelAngle=0,titleFontSize=20)),
            alt.Color('Tech_name', scale=alt.Scale(domain=cmap['Tech_name'].values.tolist(), range=cmap['Color'].values.tolist())),
            tooltip=['YEAR','Tech_name','sum(VALUE)']
        )
fe_dem

,YEAR,Tech_name,Color,VALUE
10129,2019.0,PTR-3W-CNG,Orange,0.000053
10130,2020.0,PTR-3W-CNG,Orange,0.000053
10131,2021.0,PTR-3W-CNG,Orange,0.000053
10132,2022.0,PTR-3W-CNG,Orange,0.000053
10133,2023.0,PTR-3W-CNG,Orange,0.000079
10134,2024.0,PTR-3W-CNG,Orange,0.000079
10135,2025.0,PTR-3W-CNG,Orange,0.000079
10136,2026.0,PTR-3W-CNG,Orange,0.000079
10137,2027.0,PTR-3W-CNG,Orange,0.000079
10138,2028.0,PTR-3W-CNG,Orange,0.000079


In [44]:
final_energy_all#.groupby(['YEAR','Tech_name','Color']).sum().reset_index()

,YEAR,Tech_name,Color,DAILYTIMEBRACKET,DAYTYPE,MODE_OF_OPERATION,REGION2,SCENARIO,SEASON,STORAGE,TIMESLICE,VALUE
0,2019.0,Agriculture-mecanization-Oil,Red,0.0,0.0,0.0,0.0,0,0.0,0.0,78.0,0.111300
1,2019.0,Agriculture-pump-Electricity,Blue,0.0,0.0,0.0,0.0,0,0.0,0.0,78.0,0.062500
2,2019.0,Agriculture-pump-Oil,Red,0.0,0.0,0.0,0.0,0,0.0,0.0,78.0,0.004800
3,2019.0,Agriculture-pump-Solar,Yellow,0.0,0.0,0.0,0.0,0,0.0,0.0,33.0,0.000100
4,2019.0,Commercial-Electricity,Blue,0.0,0.0,0.0,0.0,0,0.0,0.0,78.0,1.727300
5,2019.0,Commercial-Solar Water Heat,Yellow,0.0,0.0,0.0,0.0,0,0.0,0.0,33.0,0.008200
6,2019.0,Cooking rural-Biogas,Teal,0.0,0.0,0.0,0.0,0,0.0,0.0,78.0,0.003000
7,2019.0,Cooking rural-Biomass,Olive,0.0,0.0,0.0,0.0,0,0.0,0.0,78.0,2.874000
8,2019.0,Cooking rural-Biomass Improved,Olive,0.0,0.0,0.0,0.0,0,0.0,0.0,78.0,0.018000
9,2019.0,Cooking rural-Coal,Black,0.0,0.0,0.0,0.0,0,0.0,0.0,78.0,0.002000


In [37]:
fe_dem.loc[fe_dem['YEAR']==2050]

,YEAR,Tech_name,Color,VALUE
10157,2050.0,PTR-3W-CNG,Orange,0.000106
10189,2050.0,PTR-3W-CNG,Orange,0.000047
10221,2050.0,PTR-3W-CNG,Orange,0.000070
10253,2050.0,PTR-3W-CNG,Orange,0.000057
10285,2050.0,PTR-3W-CNG,Orange,0.000103
10316,2050.0,PTR-3W-CNG,Orange,0.000046
10348,2050.0,PTR-3W-CNG,Orange,0.000069
10380,2050.0,PTR-3W-CNG,Orange,0.000058
10412,2050.0,PTR-3W-CNG,Orange,0.000105
10444,2050.0,PTR-3W-CNG,Orange,0.000047
